In [1]:

import asyncio
import logging
import os
import ast
import glob
import importlib.util
from IPython import display
from typing import Dict, Callable, List

from google import genai
from google.genai import types

In [20]:
os.environ['GOOGLE_API_KEY'] = os.getenv('gemini_key')
client = genai.Client(http_options={
    'api_version': 'v1beta'
})
model_name = "gemini-2.5-pro-exp-03-25"

In [21]:
logger = logging.getLogger('Live')
# logger.setLevel('DEBUG')
logger.setLevel('INFO')

MD_HEADING = "#"

In [22]:
async def handle_tool_call(session, tool_call, custom_functions):
  for fc in tool_call.function_calls:
    function_name = fc.name
    function_args = fc.args
    # Find the function object from the list based on the function name
    tool_response = types.LiveClientToolResponse(
        function_responses=[types.FunctionResponse(
            name=fc.name,
            id=fc.id,
            response={
                'result': custom_functions[function_name](**function_args)},
        )]
    )

    print('\n>>> ', tool_response)
    await session.send(input=tool_response)

In [23]:
def handle_server_content(server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'``` python\n{executable_code.code}\n```'))
        display.display(display.Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'```\n{code_execution_result.output}\n```'))
        display.display(display.Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display.display(
        display.HTML(grounding_metadata.search_entry_point.rendered_content))

  return

In [24]:
def extract_functions_with_instruction(file_path: str) -> Dict[str, Callable]:
    with open(file_path, 'r', encoding='utf-8') as file:
        parsed = ast.parse(file.read(), file_path)

    functions = {}
    for node in ast.walk(parsed):
        if isinstance(node, ast.FunctionDef):
            params = [arg.arg for arg in node.args.args]
            module_name = os.path.splitext(os.path.basename(file_path))[0]
            spec = importlib.util.spec_from_file_location(
                module_name, file_path)
            if spec and spec.loader:
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
                func = getattr(module, node.name, None)
                if callable(func):
                    functions[node.name] = func
    return functions


def load_custom_tools(custom_dir: str = os.path.join("simple", "code", "custom")) -> Dict[str, Callable]:
    registry = {}
    py_files = glob.glob(os.path.join(custom_dir, "*.py"))
    for file in py_files:
        registry.update(extract_functions_with_instruction(file))
    return registry


def load_builtin_tools() -> Dict[str, Callable]:
    builtin_path = os.path.join("simple", "code", "function_call.py")
    return extract_functions_with_instruction(builtin_path)


def get_tool_registry() -> Dict[str, Callable]:
    registry = load_builtin_tools()
    registry.update(load_custom_tools())
    return registry


tool_registry = get_tool_registry()


In [25]:
def get_all_tools_docs():
    """Gets list of tools if"""
    return [types.FunctionDeclaration.from_callable(
        callable=func, client=client).to_json_dict()
        for name, func in tool_registry.items()]

In [26]:
def code_corpus(directory: str) -> List[str]:
    """
    Builds a corpus by reading all Python, Typescript and Javascript files in the directory.
    """
    exclude_files = []
    exclude_dirs = [
        'idea',
        'imagev1',
        'models',
        'pretrained',
        'prompts',
        'pyds',
        'node_modules',
        '.next',
        '__pycache__'
    ]
    file_paths = []
    for root, dirs, files in os.walk(directory):
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        for file in files:
            if file.split('.')[-1] in ['py', 'ts', 'tsx', 'js', 'jsx', 'md'] and file not in exclude_files:
                file_paths.append(os.path.join(root, file))

    corpus = []
    for file_path in file_paths:
        try:
            text = open(file_path, 'r', encoding='utf-8').read()
            # corpus.append(f'\n{MD_HEADING} {file_path}:\n{text}\n')
            corpus.append({file_path: text})
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    return corpus

In [27]:
code_corpus('./web/build')

[{'./web/build\\guide.py': '# Dashboard view(main screen) for Projects(sites) analytics\n# # - visitors\n# # - interactions\n# # - location\n# # - Device type\n\n\n# Crud Page for Projects\n# # Show list of all projects in site\n\n# # view project in site\n# # create new project in site\n# # delete project in site\n# # set project as deployable\n\n# # edit project in site\n# # # - edit next js backend\n# # # - edit next js frontend\n# # # - edit next js media\n# # # - edit database\n# # # - save database\n# # # - re-build project\n# # # - add saved database with updated data\n\n# Crud Page for Projects(Deployment)\n# # Show list of all projects deployed\n\n# # view project deployed\n# # download project deployed\n\n# # create new project deployment based on deployable\n# # delete project deployment\n'},
 {'./web/build\\main.py': 'import os\nimport shutil  # Needed for directory removal\nimport subprocess\nimport threading\nimport queue\nimport tkinter as tk\nfrom tkinter import ttk, si

In [28]:
tool_registry.update({code_corpus.__name__: code_corpus})

In [29]:
tool_registry

{'execute_bash_command': <function function_call.execute_bash_command(command: str) -> str>,
 'write_custom_python_file': <function function_call.write_custom_python_file(file_path: str, code: str) -> str>,
 'create_file': <function first_file_utils.create_file(file_path: str, content: str) -> str>,
 'modify_file': <function first_file_utils.modify_file(file_path: str, content: str, line_number: int) -> str>,
 'get_directory_tree': <function get_directory_tree.get_directory_tree(path: str) -> str>,
 'code_corpus': <function __main__.code_corpus(directory: str) -> List[str]>}

In [30]:
def get_tools_docs():
    """Gets list of tools that can be used"""
    base = [types.FunctionDeclaration.from_callable(
        callable=func, client=client).to_json_dict()
        for name, func in tool_registry.items() if name in ['write_custom_python_file', 'execute_bash_command', 'create_file', 'modify_file']]
    # base.append(types.FunctionDeclaration.from_callable(
    #     callable=get_tools_docs, client=client).to_json_dict())
    base.append(types.FunctionDeclaration.from_callable(
        callable=code_corpus, client=client).to_json_dict())

    return base

In [31]:
get_tools_docs()

[{'description': '\nExecutes a bash command provided as a string.\n\nArgs:\n    command: The command to execute.\n             - If string: Treated as a single command line, split by shlex.\n\nReturns:\n    str: The standard output of the command if successful.\n\nRaises:\n    ValueError: If the command format is unsupported or invalid.\n    Exception: If the command fails to execute (non-zero exit code or other error).\n              The exception message includes stderr.\n',
  'name': 'execute_bash_command',
  'parameters': {'properties': {'command': {'type': 'STRING'}},
   'required': ['command'],
   'type': 'OBJECT'}},
 {'description': '\nWrites the provided Python code to a file within the \'simple/code/custom\' directory.\nPrevents path traversal attacks.\n\nArgs:\n    file_path (str): The desired filename (e.g., "my_tool.py"). Should not contain path separators.\n    code (str): The Python code content to write.\n\nReturns:\n    str: The full path to the written file if successf

In [32]:
n = 0
custom_functions = tool_registry

async def run(prompt, modality="TEXT", tools=None):

    global n

    if tools is None:

        tools = []

    config = {


        "tools": tools,


        "response_modalities": [modality]}

    async with client.aio.live.connect(model=model_name, config=config) as session:

        display.display(display.Markdown(prompt))

        display.display(display.Markdown('-------------------------------'))

        await session.send(input=prompt, end_of_turn=True)

        async for response in session.receive():

            logger.debug(str(response))

            if text := response.text:

                display.display(display.Markdown(text))

                continue

            server_content = response.server_content

            if server_content is not None:

                handle_server_content(server_content)

                continue

            tool_call = response.tool_call

            if tool_call is not None:

                await handle_tool_call(session, tool_call, custom_functions)

        n = n+1

In [33]:
tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': get_tools_docs()}
]
corpus = code_corpus('./web/build')

user_request = "can you add the different departments"

user_prompt = f"""\n\n# Request:\n{user_request}"""

prompt = f'''# Considering the following:\n{corpus}{user_prompt}'''

await run(prompt, tools=tools, modality="TEXT")

InvalidStatus: server rejected WebSocket connection: HTTP 404